In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the two datasets
data1 = pd.read_csv('/Users/ernestgaisie/Desktop/Housing Prices Project/HouseListings-Top45Cities-10292023-kaggle.csv', encoding='latin1')
data2 = pd.read_csv('/Users/ernestgaisie/Desktop/Housing Prices Project/ca_real_estate 2.csv', encoding='latin1')

# Display basic info for both datasets
print("Data1 Info:")
print(data1.info())

print("Data2 Info:")
print(data2.info())

Data1 Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35768 entries, 0 to 35767
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   City                  35768 non-null  object 
 1   Price                 35768 non-null  float64
 2   Address               35768 non-null  object 
 3   Number_Beds           35768 non-null  int64  
 4   Number_Baths          35768 non-null  int64  
 5   Province              35768 non-null  object 
 6   Population            35768 non-null  int64  
 7   Latitude              35768 non-null  float64
 8   Longitude             35768 non-null  float64
 9   Median_Family_Income  35768 non-null  float64
dtypes: float64(4), int64(3), object(3)
memory usage: 2.7+ MB
None
Data2 Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 

In [3]:
# Mapping for province names
province_mapping = {
    'ON': 'Ontario',
    'QC': 'Quebec',
    'BC': 'British Columbia',
    'AB': 'Alberta',
    'MB': 'Manitoba',
    'SK': 'Saskatchewan',
    'NS': 'Nova Scotia',
    'NB': 'New Brunswick',
    'NL': 'Newfoundland and Labrador',
    'PE': 'Prince Edward Island',
    'YT': 'Yukon',
    'NT': 'Northwest Territories',
    'NU': 'Nunavut'
}

# Replace province abbreviations with full names
data1['Province'] = data1['Province'].replace(province_mapping)
data2['Province'] = data2['Province'].replace(province_mapping)

In [4]:
# Rename columns to align datasets
rename_map_data1 = {'Number_Beds': 'Bedrooms', 'Number_Baths': 'Bathrooms', 'Price': 'Price_x'}
rename_map_data2 = {'Price': 'Price_y'}

data1.rename(columns=rename_map_data1, inplace=True)
data2.rename(columns=rename_map_data2, inplace=True)

In [5]:
# Set Type column to 'House' for data1
data1['Type'] = 'House'

In [6]:
# Identify missing columns in each dataset
missing_columns_data1 = set(data2.columns) - set(data1.columns)
missing_columns_data2 = set(data1.columns) - set(data2.columns)

# Add missing columns with NaN
for col in missing_columns_data1:
    data1[col] = np.nan
for col in missing_columns_data2:
    data2[col] = np.nan

In [7]:
# Append datasets
combined_data = pd.concat([data1, data2], ignore_index=True)

# Display combined dataset info
print("Combined Dataset Info:")
print(combined_data.info())

Combined Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40768 entries, 0 to 40767
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   City                  40768 non-null  object 
 1   Price_x               35768 non-null  float64
 2   Address               35768 non-null  object 
 3   Bedrooms              40768 non-null  int64  
 4   Bathrooms             40768 non-null  int64  
 5   Province              40768 non-null  object 
 6   Population            35768 non-null  float64
 7   Latitude              35768 non-null  float64
 8   Longitude             35768 non-null  float64
 9   Median_Family_Income  35768 non-null  float64
 10  Type                  40768 non-null  object 
 11  Garage                5000 non-null   float64
 12  Year_Built            5000 non-null   float64
 13  Price_y               5000 non-null   float64
 14  Lot_Area              5000 non-null   float64
 

In [8]:
# Step 1: Create a mapping of City to Population from data1
city_population_map = data1.set_index('City')['Population'].to_dict()

# Step 2: Infer Population for missing values in combined_data
combined_data['Population'] = combined_data.apply(
    lambda row: city_population_map.get(row['City'], row['Population']), axis=1
)

# Step 3: Handle remaining missing Population values using grouped median
if combined_data['Population'].isnull().sum() > 0:
    combined_data['Population'] = combined_data.groupby(['Province', 'City'])['Population'].transform(
        lambda x: x.fillna(x.median())
    )

# Step 4: Final fallback to overall median if Population is still missing
if combined_data['Population'].isnull().sum() > 0:
    combined_data['Population'].fillna(combined_data['Population'].median(), inplace=True)

# Verify the results
print("Remaining missing Population values:", combined_data['Population'].isnull().sum())

Remaining missing Population values: 0


In [9]:
# Combine Price_x and Price_y into a single column
combined_data['Price'] = combined_data['Price_x'].combine_first(combined_data['Price_y'])
combined_data.drop(['Price_x', 'Price_y'], axis=1, inplace=True)

In [10]:
# Columns to infer
columns_to_infer = ['Lot_Area', 'Median_Family_Income']

# Grouped inference
for col in columns_to_infer:
    if col in combined_data.columns:
        combined_data[col] = combined_data.groupby(['City', 'Province', 'Bedrooms'])[col].transform(
            lambda x: x.fillna(x.median())
        )

# Fallback to broader grouping (Province-level)
for col in columns_to_infer:
    if col in combined_data.columns and combined_data[col].isnull().sum() > 0:
        combined_data[col] = combined_data.groupby(['Province', 'Bedrooms'])[col].transform(
            lambda x: x.fillna(x.median())
        )

In [11]:
# No changes needed for Garage and Year_Built; leave NaN
print("Non-numeric columns left as NaN:", combined_data.select_dtypes(exclude=['float64', 'int64']).columns)

Non-numeric columns left as NaN: Index(['City', 'Address', 'Province', 'Type'], dtype='object')


In [12]:
# Drop unnecessary columns
combined_data.drop(['Latitude', 'Longitude'], axis=1, inplace=True)

In [13]:
# Save the cleaned dataset
combined_data.to_csv('combined_data_cleaned.csv', index=False)
print("Final cleaned dataset has been saved as 'combined_data_cleaned.csv'.")

Final cleaned dataset has been saved as 'combined_data_cleaned.csv'.


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [15]:
# Load the combined dataset (already preprocessed in previous steps)
combined_data = pd.read_csv('combined_data_cleaned.csv')

# Display basic information for verification
print("Dataset Info:")
print(combined_data.info())

In [21]:
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X.select_dtypes(include=['object'])

print("Numerical Columns:", numerical_cols)
print("Categorical Columns:", categorical_cols)

Numerical Columns: Index(['Bedrooms', 'Bathrooms', 'Population', 'Median_Family_Income', 'Garage',
       'Year_Built', 'Lot_Area', 'SqFt'],
      dtype='object')
Categorical Columns:             City                  Address Province       Type
0        Toronto    #318 -20 SOUTHPORT ST  Ontario      House
1        Toronto    #818 -60 SOUTHPORT ST  Ontario      House
2        Toronto  #714 -859 THE QUEENSWAY  Ontario      House
3        Toronto         275 MORTIMER AVE  Ontario      House
4        Toronto    #420 -388 RICHMOND ST  Ontario      House
...          ...                      ...      ...        ...
40763     Ottawa                      NaN  Ontario  Apartment
40764   Montreal                      NaN  Ontario  Apartment
40765     Ottawa                      NaN  Alberta      House
40766     Ottawa                      NaN  Ontario      Condo
40767  Vancouver                      NaN  Alberta      Condo

[40768 rows x 4 columns]


In [22]:
# Step 3: Preprocessing for numerical and categorical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())  # Standardize numerical features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

In [23]:
# Step 4: Combine preprocessors into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
# Step 6: Visualize feature relationships with target
# Correlation matrix
plt.figure(figsize=(12, 8))
correlation_matrix = combined_data.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

ValueError: could not convert string to float: 'Toronto'

<Figure size 1200x800 with 0 Axes>